In [ ]:
import numpy
numpy.set_printoptions(threshold=numpy.inf)

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 正規化
! pip install neologdn

# sudachiをインストール
! pip install sudachipy
! pip install sudachidict_core #単語分割用の辞書 core->full

# janomeをインストール
! pip install janome! pip install janome

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
with open('/content/drive/MyDrive/exp5-3_japanese/stopwords.txt', 'r') as f:
    stopword = f.read().split("\n")
    stopword.append('、')
    stopword.append('。')
stopword[:10]

['の', 'に', 'は', 'を', 'た', 'が', 'で', 'て', 'と', 'し']

In [ ]:
with open('/content/drive/MyDrive/exp5-3_japanese/test.text.txt', 'r') as f:
    train_text = f.read().split("\n")
train_text = train_text[:-1]
print(train_text)

import neologdn

for i in range(len(train_text)):
    train_text[i] = neologdn.normalize(train_text[i])

['レンガの建物の前を、乳母車を押した女性が歩いています。\t厩舎で馬と女性とが寄り添っています。', '山の上に顔の白い牛が2頭います。\t曇り空の山肌で、牛が２匹草を食んでいます。', 'バナナを持った人が道路を通行しています。\t道の上をバナナを背負った男性が歩いています。', 'スケートボーダーが手すりを滑っています。\t階段の手すりでスケートボードをする男性がいます。', 'ダブルベッドの上で、女性が足を組み横たわっています。\tベッドの上に寝転んで、足を組んでいる人が映っています。', '１頭のシマウマが外で立ち止まっている。\t一頭のシマウマが日陰に立っています。', 'テニスコートで男性がラケットを振っています。\tコートの上で男性がラケットを両手に持って立っています。', '三頭の馬が牧場で草を食んでいます。\tバナナやキウイなどが器に盛られています。', 'ベッドの上で衣装を着た二人の子供が立っています。\t路上にバイクに乗った人の集団がいます。', 'ヘルメットを被ったスキーヤーが雪の上を滑っています。\tスキーをしている赤いスキーウェアを着た子供がいます。', '大きなタイル敷きのバスルームで、白い大きな浴槽の様子。\tバスルームに絵画が飾られています。', 'オレンジにカミソリの刃が一枚刺さり、ビンと金属の容器が置いてあります。\tオレンジ色の果物の皮にカミソリの刃が突き刺さっています。', 'テーブルの上には小さな置時計があります\tテーブルの上に時計が置かれています。', '頭の上にあるボールを、男性が見上げています。\t頭の上にある鳥はボールを見ている', '白い皿の上に焼いた丸いピザがのせてあります。\t大きくて丸い焼いたピザが、白い皿の上にのせてあります。', 'テーブルの上に、ピザが置かれています。\tピザを乗せた皿が置かれています。', '建物前の雪の地面に大人のスキーヤーと子供のスキーヤーがいます。\t子供がスキー板をはいて立っています。', 'バイクに乗った警官が一列になって道路を走っています。\t蛍光イエローのジャンパーを着た警察官達が道路の警備をしています。', '遠くに山合いの見える場所に飛行機が飛んでいます。\t競技場でサッカーをしている人達がいます。', 'ジャンプ台から男性スケートボーダーがジャンプしています。

In [ ]:
# sudachiによる正規化を行う関数を定義
import sudachipy
from sudachipy import Dictionary
from sudachipy import SplitMode
tokenizer = Dictionary().create()

def normalize_sud(text, mode):
    mode = {
        'A': sudachipy.Tokenizer.SplitMode.A,
        'B': sudachipy.Tokenizer.SplitMode.B,
        'C': sudachipy.Tokenizer.SplitMode.C}[mode]
    words = list()
    
    #tokens = [token.normalized_form() for token in tokenizer.tokenize(text), mode)]
    for token in tokenizer.tokenize(text, mode):
        word = token.surface()
        pos = " ".join(token.part_of_speech())
        if word.isnumeric():
            word = '0'
            words.append(word)
        else:
            if pos[0] != '助' and  pos[0] != '補':
                if not word in stopword:
                    words.append(token.normalized_form())
    return words

In [ ]:
from janome.tokenizer import Tokenizer

def normalize_ja(text):
    words = list()
    for token in Tokenizer().tokenize(text):
        word = token.surface
        pos = token.part_of_speech
        if word.isnumeric():
            word = '0'
            words.append(word)
        else:
            #if pos[0] != '助' and  pos[0] != '補'and pos[0] != '記':
            if pos[0] != '記':
                words.append(word)
    return words

In [ ]:
textA1 = list()
textA2 = list() 
textB1 = list()
textB2 = list()
textC1 = list()
textC2 = list()
textja1 = list()
textja2 = list()

for t_text in train_text:
    txt1, txt2 = t_text.split("\t")
    textA1.append(normalize_sud(txt1, 'A')) # text1について正規化 & 単語分割
    textA2.append(normalize_sud(txt2, 'A'))

    textB1.append(normalize_sud(txt1, 'B')) 
    textB2.append(normalize_sud(txt2, 'B'))

    textC1.append(normalize_sud(txt1, 'C')) 
    textC2.append(normalize_sud(txt2, 'C'))

In [ ]:
! curl -Lo chive-1.2-mc5_gensim.tar.gz https://sudachi.s3-ap-northeast-1.amazonaws.com/chive/chive-1.2-mc5_gensim.tar.gz
! tar -zxvf chive-1.2-mc5_gensim.tar.gz

import gensim

w2v = gensim.models.KeyedVectors.load("./chive-1.2-mc5_gensim/chive-1.2-mc5.kv")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3464M  100 3464M    0     0  16.8M      0  0:03:25  0:03:25 --:--:-- 17.2M
chive-1.1-mc5_gensim/
chive-1.1-mc5_gensim/LICENSE
chive-1.1-mc5_gensim/README.md
chive-1.1-mc5_gensim/chive-1.1-mc5.kv
chive-1.1-mc5_gensim/chive-1.1-mc5.kv.vectors.npy


In [ ]:
# BoW

# 語彙（扱いたい全ての単語）
vocab = set()
for text1_ in textA1:
    for word in text1_:
        vocab.add(word)

for text2_ in textA2:
    for word in text2_:
        vocab.add(word)

# 各単語にIDを割り当てる 日課：0番目, ...
word2id = dict()
for word in vocab:
    word2id[word] = len(word2id)

def vectorizebow(words, vocab, word2id):
    vector = np.zeros(len(vocab))
    for word in words:
        if word in vocab:
            vector[word2id[word]] = 1
            #vector[word2id[word]] = idf(word)
    return vector

In [ ]:
import math
# 辞書(回数)作成のための わかち
def wakati(text, mode):
    mode = {
        'A': sudachipy.Tokenizer.SplitMode.A,
        'B': sudachipy.Tokenizer.SplitMode.B,
        'C': sudachipy.Tokenizer.SplitMode.C}[mode]
    return [token.normalized_form() for token in tokenizer.tokenize(text, mode)]


# 頻度とidf
from collections import defaultdict
freq_dict_a = defaultdict(int) #頻度
prob_dict_a = defaultdict(float) #確率
freq_dict_b = defaultdict(int) #頻度
prob_dict_b = defaultdict(float) #確率
freq_dict_c = defaultdict(int) #頻度
prob_dict_c = defaultdict(float) #確率

with open('/content/drive/MyDrive/exp5-3_japanese/ja_wiki.sent.10m.txt', 'r') as f:
    for i, line in enumerate(f):
        #if i == 800000:
            #break
        wakatitext = wakati(line, 'A')
        for word in wakatitext:
            prob_dict_a[word] += 1
        for word in set(wakatitext):
            freq_dict_a[word] += 1
        
        wakatitext = wakati(line, 'B')
        for word in wakatitext:
            prob_dict_b[word] += 1
        for word in set(wakatitext):
            freq_dict_b[word] += 1

        wakatitext = wakati(line, 'C')
        for word in wakatitext:
            prob_dict_c[word] += 1
        for word in set(wakatitext):
            freq_dict_c[word] += 1

In [ ]:
 #idfの計算
def idf(word, dic):
     if dic[word] == 0:
         return 1
     else:
         return 1 + math.log( 10000000 / dic[word])

In [ ]:
#ベクトル中心化　* 確率（の計算)
import numpy as np
def centvec(dic):
    total = sum(dic.values())
    for word, freq in dic.items(): # 辞書からkeyとvalueを取り出し
        dic[word] = freq / total
    
    vocab = w2v.wv.index2word
    meanvec = np.zeros(w2v.vector_size)
    for voc in set(dic.keys()) & set(vocab):
        meanvec +=  w2v[voc] * dic[voc]
    return meanvec

meanvec_a = centvec(prob_dict_a)
meanvec_b = centvec(prob_dict_b)
meanvec_c = centvec(prob_dict_c)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [ ]:
# 単語リストをベクトル化するための関数を定義
def vectorize(words, w2v, meanvec, idfdic):
    vectors = list()
    for word in words:
        #word = clean(word) #クリーニング
        if word in w2v:
            vectors.append((w2v[word]-meanvec)* idf(word, idfdic)) # ベクトル中心化 とidf
    # vec = ( np.array(vectors).max(axis=0)*1 + np.array(vectors).mean(axis=0)*1 + np.array(vectors).min(axis=0)*0)
    vec = np.array(vectors).mean(axis=0)
    return vec

# 類似度推定: 2つのベクトル -> コサイン類似度
def cos(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [ ]:
y_a = list()
y_b = list()
y_c = list()
y_bow = list()
y_ja = list()

y_a2 = list()
y_b2 = list()
y_c2 = list()
for i in range(len(textA1)):
    y_a.append('{:f}'.format(cos(vectorize(textA1[i], w2v, meanvec_a, freq_dict_a), vectorize(textA2[i], w2v, meanvec_a, freq_dict_a)))) # yに追加
    # y_a2.append('{:f}'.format(cos(vectorize(textA1[i], w2v, 0, freq_dict_a), vectorize(textA2[i], w2v, 0, freq_dict_a)))) # yに追加
    
    y_b.append('{:f}'.format(cos(vectorize(textB1[i], w2v, meanvec_b, freq_dict_b), vectorize(textB2[i], w2v, meanvec_b, freq_dict_c))))
    # y_b2.append('{:f}'.format(cos(vectorize(textB1[i], w2v, 0, freq_dict_b), vectorize(textB2[i], w2v, 0, freq_dict_b))))
    
    y_c.append('{:f}'.format(cos(vectorize(textC1[i], w2v, meanvec_c, freq_dict_c), vectorize(textC2[i], w2v, meanvec_c, freq_dict_c))))
    # y_c2.append('{:f}'.format(cos(vectorize(textC1[i], w2v, 0, freq_dict_c), vectorize(textC2[i], w2v, 0, freq_dict_c))))
    
    y_bow.append('{:f}'.format(cos(vectorizebow(textA1[i], vocab, word2id), vectorizebow(textA2[i], vocab, word2id))))
    # y_ja.append('{:f}'.format(cos(vectorize(textja1[i], w2v, 0), vectorize(textja2[i], w2v, 0))))

In [ ]:
y = list()
for i in range(len(y_a)):
    avg = (float(y_a[i])*1 + float(y_b[i])*0.0+ float(y_c[i])*1 + float(y_bow[i])*0.0)
    y.append('{:f}'.format(avg))
print(y)

with open('/content/drive/MyDrive/exp5-3_japanese/ja.test.label.txt', 'w') as f:
    for i, label in enumerate(y):
        if i == len(y)-1:
            f.write(label)
        else:
            f.write(label+'\n')
print(len(y))

['0.851138', '1.348027', '1.207795', '0.747391', '1.509582', '1.590977', '1.547479', '0.643832', '0.538616', '1.192162', '0.660733', '1.453829', '1.629699', '1.552840', '1.961800', '1.691650', '1.117132', '1.059409', '0.120904', '1.512077', '2.000000', '1.721236', '1.801267', '0.549255', '1.474925', '1.400251', '0.834167', '1.374604', '0.708922', '1.259647', '1.501649', '1.445470', '1.384534', '2.000000', '1.555976', '0.855925', '0.772020', '1.452839', '1.506810', '0.521257', '1.370255', '0.906002', '1.025158', '1.538847', '1.884185', '1.838656', '1.394298', '1.212616', '0.991490', '1.119080', '1.157065', '0.548551', '1.251001', '1.871571', '1.684180', '1.724690', '1.671726', '1.262754', '1.391780', '1.461092', '1.293892', '1.082396', '0.574862', '1.800278', '1.509601', '1.625879', '0.267933', '1.526265', '1.218038', '1.285841', '1.346384', '1.447450', '1.682433', '1.079221', '1.866776', '1.580550', '0.797489', '1.936701', '1.441852', '1.106122', '1.182940', '1.431122', '1.699231', '1.

#スコアメモ

a中心化   : a
* 1 : 0  : 0.812
* 0.9 : 0.1  : 0.812
* 2 : 0.5  : 0.812
* 1: 1  :  0.810
* 0.3 : 0.7  : 0.807
* 0.9 : 0.1  : 0.812
* 0.8 : 0.2  : 0.812
* 0.7 : 0.3  : 0.811
* 0.6 : 0.4  : 0.811


b中心化   : b 
* 1: 1  :  0.808
* 1: 0.5  :  0.809
* 0.5: 1  :  0.807


c中心化   : c  
* 1: 1  :  0.813
* 1: 0.5  :  0.813
* 2: 0.5  : 0.813

ベクトル中心化, idfなし

* 2 : 0.5 : 3 : 0.3 : 0.3 ： 0.815
* 3 : 0.5 : 3 : 0.3 : 0.3 ： 0.815
* 3 : 0.5 : 3 : 0.3 : 1 ： 0.815
* 5 : 0.5 : 5 : 0.3 : 0.3 ： 0.815
* 1 : 1 : 1 : 1 : 1 ： 0.815
* 1 : 1 : 1 : 0.5 : 1 ： 0.812
* 1 : 1 : 1 : 0 : 1 ： 0.813
* 1 : 1 : 2 : 1 : 1 ： 0.812

2：0.5：2：0.3：0.3 + ベクトル中心化

* idfあり：0.810
* idfなし：0.815

sudachi A：B：C + 正規化済みストップワード : Bow：janome

*   2.0 : 0.5 : 2.0 : 0.3 : 0.3 ：  0.798

sudachi A：B：C + 助詞,補助除去 + 正規化済みストップワード : Bow：janome

*   2.0 : 0.5 : 2.0 : 0.3 : 0.3 ：  0.808
*   2.0 : 0.5 : 2.0 : 0.3 : 0.4 ：  0.808
*   2.0 : 0.5 : 2.0 : 0.3 : 0.5 ：  0.807

sudachi A：B：C + 助詞,補助除去 + 正規化済みストップワード

*   1.0 : 1.0 : 1.0 ：  0.806
*   1.0 : 1.0 : 2.0 ：  0.806
*   2.0 : 0.5 : 2.0 ：  0.807

sudachi A：B：C + 助詞,補助除去 + ストップワード : Bow：janome

*   1.0 : 1.0 : 1.0 : 1.0 : 1.0 ：  0.801
*   2.0 : 0.5 : 2.0 : 0.3 : 0.3 ：  0.807
*   2.0 : 0.5 : 2.0 : 0.3 : 0.4 ：  0.807
*   2.0 : 0.5 : 2.0 : 0.3 : 0.5 ：  0.807
*   2.0 : 0.5 : 2.0 : 0.3 : 1.0 ：  0.807
*   2.0 : 0.5 : 2.0 : 0.3 : 2.0 ：  0.803

sudachi A：B：C + 助詞,補助除去 + ストップワード : Bow

*   1.0 : 1.0 : 1.0 : 1.0 ：  0.800
*   1.0 : 1.0 : 1.0 : 0.1 ：  0.806
*   1.0 : 0.5 : 1.0 : 0.1 ：  0.806
*   1.0 : 0.5 : 1.0 : 0.2 ：  0.807
*   1.0 : 0.5 : 1.0 : 0.3 ：  0.807
*   1.0 : 0.5 : 1.0 : 0.4 ：  0.806
*   1.0 : 0.5 : 1.5 : 0.3 ：  0.807
*   2.0 : 0.5 : 2.0 : 0.3 ：  0.807



sudachi A：B：C + 助詞,補助除去 + ストップワード

*   1.0 : 1.0 : 1.0 ：  0.805
*   1.0 : 1.0 : 2.0 ：  0.805
*   2.0 : 1.0 : 1.0 ：  0.805
*   2.0 : 1.0 : 2.0 ：  0.805

sudachi A：B：C + 助詞,補助除去

*   1.0 : 1.0 : 1.0 ：  0.798
*   1.0 : 0.5 : 1.0 ：  0.798

sudachi A：B：C + ストップワード使用

*   1.0 : 1.0 : 1.0 ：  0.804
*   0.5 : 1.0 : 1.0 ：  0.804
*   1.0 : 0.5 : 1.0 ：  0.805
*   1.0 : 1.0 : 0.5 ：  0.804

*   1.5 : 0.5 : 1.0 ：  0.805

Sudachi同義語辞書でカテゴライズ：0.761

Sudachi C：BOW

*   0.5:0.5  ：0.764
*   0.6:0.4  ：0.777
*   0.7:0.3  ：0.788
*   0.8:0.2  ：0.794
*   0.85:0.15  ：0.796
*   0.87:0.13  ：0.798
*   0.88:0.12  ：0.799
*   0.89:0.11  ：0.799
*   0.9:0.1  ：0.798
*   0.91:0.09  ：0.798
*   0.95:0.05  ：0.796

*   y：pos[0] ==sudachi C ( '名' or pos[0] == '形'or pos[0] == '動' かつ　数字をすべて0に)
*   y2：bow
*   y3：sudachi C (pos[0] == '形' or pos[0] == '名'かつ　数字をすべて0に)
 
*   0.45:0.02:0.58 ：0.803
*   0.58:0.02:0.50 ：0.804
*   0.63:0.02:0.45 ：0.804
*   0.70:0.02:0.28 ：0.803
*   0.89:0.03:0.08 ：0.800
*   0.90:0.04:0.06 ：0.800
*   0.90:0.03:0.07 ：0.801
*   0.90:0.02:0.08 ：0.800
*   0.90:0.01:0.09 ：0.799


*   y：sudachi C (pos[0] == '名' or pos[0] == '形'or pos[0] == '動' かつ　数字をすべて0に)
*   y2：bow
*   y3：sudachi A (pos[0] == '形' or pos[0] == '名'かつ　数字をすべて0に)
 
*   0.45:0.02:0.58 ：0.779
*   0.58:0.02:0.50 ：0.779
*   0.70:0.02:0.28 ：0.779

*   y：sudachi C (pos[0] == '名' or pos[0] == '形'or pos[0] == '動' かつ　数字をすべて0に)
*   y2：bow
*   y3：sudachi C 
(pos[0] == '形' or pos[0] == '名'かつ　数字をすべて0に)
*   y4：sudachi B (pos[0] == '動' or pos[0] == '名'かつ　数字をすべて0に)

*   0.05:0.02:0.44:0.49 ：0.806
*   0.08:0.02:0.45:0.45 ：0.807
*   0.08:0.02:0.50:0.40 ：0.806
*   0.08:0.02:0.40:0.50 ：0.806
*   0.10:0.02:0.20:0.68 ：0.804
*   0.16:0.03:0.31:0.50 ：0.805
*   0.16:0.02:0.32:0.50 ：0.805
*   0.20:0.02:0.28:0.50 ：0.805
*   0.26:0.02:0.22:0.50 ：0.805
*   0.28:0.02:0.30:0.40 ：0.804
*   0.28:0.02:0.20:0.50 ：0.804
*   0.53:0.02:0.22:0.23 ：0.804
*   0.63:0.02:0.22:0.13 ：0.804

* ストップワード除去のみ：0.774
* sudachi用ストップワード除去のみ：0.797

*   0.08:0.10:0.42:0.5:0.3  :0.807
*   0.08:0.15:0.42:0.5:0.3  :0.807
*   0.08:0.20:0.42:0.5:0.3  :0.807
*   0.08:0.30:0.42:0.5:0.3  :0.805
*   0.08:0.20:0.42:0.7:0.3  :0.806
*   0.08:0.20:0.42:0.6:0.3  :0.807
*   0.28:0.20:0.42:0.6:0.3  :0.807


#延長戦

A+C 
* chiVe 1.2 mc5 :  0.816
* chiVe 1.2 mc90 :  0.812
* chiVe 1.1 mc5 :  0.814
* dependency-based-japanese-word-embeddings: 0.662

朝日新聞
* glove-retrofitting.bin :  0.684
* cbow.bin :  0.741
* skipgram-retrofitting.bin :  0.753
* skipgram.bin :  0.763
* glove.bin :  0.723
* cbow-retrofitting.bin :  0.747

*  WikiEntVec 使い方わからず(解凍


ベクトル化の比率： max:mean:min = 1.0 : 3.0 : 0.7
* sudachi A 中心化あり：0.815
* sudachi A 中心化なし：0.808
* sudachi B 中心化あり：0.812
* sudachi B 中心化なし：0.806
* sudachi C 中心化あり：0.817
* sudachi C 中心化なし：0.811

ベクトル化の比率： max:mean:min = 1.0 : 0.0 : 0.0
* sudachi A 中心化あり：0.687
* sudachi A 中心化なし：0.692
* sudachi B 中心化あり：0.694
* sudachi B 中心化なし：0.698
* sudachi C 中心化あり：0.698
* sudachi C 中心化なし：0.703

ベクトル化の比率： max:mean:min = 0.0 : 1.0 : 0.0
* sudachi A 中心化あり：0.812
* sudachi A 中心化なし：0.802
* sudachi B 中心化あり：0.808
* sudachi B 中心化なし：0.800
* sudachi C 中心化あり：0.812
* sudachi C 中心化なし：0.804

* sudachi A 中心化あり idf：0.813
* sudachi A 中心化なし idf：0.809
* sudachi B 中心化あり idf：0.811
* sudachi B 中心化なし idf：0.808
* sudachi C 中心化あり idf：0.813
* sudachi C 中心化なし idf：0.810

* sudachi A bow：0.708
* sudachi A bow idf：0.714
* sudachi B bow：0.699
* sudachi B bow idf：0.706
* sudachi C bow：0.693
* sudachi C bow idf：0.699


ベクトル化の比率： max:mean:min = 0.0 : 0.0 : 1.0
* sudachi A 中心化あり：0.684
* sudachi A 中心化なし：0.688
* sudachi B 中心化あり：0.692
* sudachi B 中心化なし：0.695
* sudachi C 中心化あり：0.694
* sudachi C 中心化なし：0.698

アンサンブル A B C ともにそれぞれのidf 中心化
* A : B : C : A bow idf_a

* 1.0 : 1.0 : 1.0 : 1.0  : 0.812
* 0.5 : 1.0 : 1.0 : 1.0  : 0.810
* 1.0 : 0.5 : 1.0 : 1.0  : 0.810
* 1.0 : 1.0 : 0.5 : 1.0  : 0.810
* 1.0 : 1.0 : 1.0 : 0.5  : 0.815
* 1.0 : 1.0 : 1.0 : 0.1  : 0.815
* 1.0 : 1.0 : 1.0 : 0.0  : 0.815

* 2.0 : 1.0 : 1.0 : 0.0  : 0.815
* 1.0 : 2.0 : 1.0 : 0.0  : 0.814
* 1.0 : 1.0 : 2.0 : 0.0  : 0.815

* 1.0 : 0.5 : 1.0 : 0.0  : 0.815
* 1.0 : 0.3 : 1.0 : 0.0  : 0.815

* 1.0 : 0.0 : 1.0 : 0.0  : 0.816
* 2.0 : 0.0 : 1.0 : 0.0  : 0.815
* 1.0 : 0.0 : 2.0 : 0.0  : 0.815


ベクトル化の比率
1.0 : 0.0 : 1.0 : 0.0
* max:mean:min = 1.0 : 3.0 : 0.7  : 0.813
* max:mean:min = 1.0 : 1.0 : 1.0  : 0.806
* max:mean:min = 1.0 : 1.0 : 0.2  : 0.806
* max:mean:min = 1.0 : 1.0 : 0.0  : 0.806
* max:mean:min = 1.0 : 2.0 : 0.0  : 0.796
* max:mean:min = 2.0 : 1.0 : 1.0  : 0.714